In [3]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [4]:
from glob import glob

In [5]:
# create data frame from the labels csv
df = pd.read_csv('./labeling/image_labels.csv')

In [6]:
df.head(3)

,filepath,xmin,xmax,ymin,ymax
0,./annotations/Cars0.xml,226,419,125,173
1,./annotations/Cars1.xml,134,262,128,160
2,./annotations/Cars10.xml,140,303,5,148


In [10]:
imgs_path = glob('./labeling/images/*.png')

In [11]:
imgs_path.sort()

In [12]:
df['imgpath'] = imgs_path

<h1><b>Data Preprocessing</b></h1>

In [13]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array # type: ignore

In [14]:
labels_array = df[['xmin','xmax','ymin','ymax']].values

In [16]:
def preprocessing_img(path): 
    load_image = load_img(path,target_size=(224,224))
    load_image_arr = img_to_array(load_image) 
    return load_image_arr

In [17]:
def normalize_img(img, img_label, load_image_arr):
    
    norm_load_image_arr = load_image_arr/255.0
    
    img_arr = cv2.imread(img)
    h, w, d = img_arr.shape
    
    xmin, xmax, ymin, ymax = img_label
    nxmin, nxmax, nymin, nymax = xmin/w, xmax/w, ymin/h, ymax/h
    
    norm_img_label = (nxmin, nxmax, nymin, nymax)
    
    return norm_load_image_arr, norm_img_label

In [18]:
data = []
labels_norm = []

for ind in range(len(df)):
    img_path = df['imgpath'][ind]
    img_label = labels_array[ind]

    load_img_arr = preprocessing_img(img_path)
    
    norm_load_image_arr, norm_img_label = normalize_img(img_path, img_label, load_img_arr)
    
    data.append(norm_load_image_arr)
    labels_norm.append(norm_img_label)
    
    

In [19]:
df['norm_load_img_arr'] = data

In [20]:
df['labels_norm'] = labels_norm

<h1>Split train and test</h1>

In [21]:
X = np.array(list(df['norm_load_img_arr']),dtype=np.float32)
y = np.array(list(df['labels_norm']),dtype=np.float32)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, random_state=42)

<h1>Deep Learning Model</h1>

In [23]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2 # type: ignore
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input # type: ignore
from tensorflow.keras.models import Model # type: ignore

import tensorflow as tf

In [24]:
inception_resnet = InceptionResNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

inception_resnet.trainable=False

In [25]:
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)

headmodel = Dense(64, activation='relu')(headmodel)
headmodel = Dense(4, activation='sigmoid')(headmodel)

In [26]:
model = Model(inputs=inception_resnet.input, outputs=headmodel)

In [27]:
model.compile(loss='mean_squared_error',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6))
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 56,794,660 (216.65 MB)

 Trainable params: 2,457,924 (9.38 MB)

 Non-trainable params: 54,336,736 (207.28 MB)

Training the model

In [28]:
from tensorflow.keras.callbacks import TensorBoard # type: ignore

In [29]:
tfb = TensorBoard(log_dir='logs_object_detection')

In [30]:
history = model.fit(x=X_train,y=y_train, batch_size=10, epochs=100, validation_data=(X_test,y_test), callbacks=[tfb])

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 24s 495ms/step - loss: 0.0699 - val_loss: 0.0481
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 437ms/step - loss: 0.0491 - val_loss: 0.0406
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 442ms/step - loss: 0.0415 - val_loss: 0.0373
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 437ms/step - loss: 0.0405 - val_loss: 0.0352
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 440ms/step - loss: 0.0343 - val_loss: 0.0334
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 441ms/step - loss: 0.0311 - val_loss: 0.0317
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 443ms/step - loss: 0.0259 - val_loss: 0.0309
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 445ms/step - loss: 0.0277 - val_loss: 0.0298
Epoch 9/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 450ms/step - loss: 0.0241 - val_loss: 0.0289
Epoch 10/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 451ms/step - loss: 0.0242 - val_loss: 0.0284
Epoch 11/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 16s 455ms/step - loss: 0.0209 - val_loss: 0.0274
Epoch 12/100
35/35 ━━━━━━━━━━━

In [31]:
# history = model.fit(x=X_train,y=y_train, batch_size=10, epochs=200, validation_data=(X_test,y_test), callbacks=[tfb],initial_epoch=100)

In [32]:
model.save('./models/license_detection.keras')